In [1]:
import pandas as pd
import glob
import csv
import os
import sys
from contextlib import contextmanager
import re
import numpy as np

In [12]:
script_path = os.getcwd()
file_path = r".\Data"
extension = "csv"

print(os.getcwd())

C:\Users\matti\MAIniacs


In [3]:
# Setting up a context manager in order to change to the data folder, create a dictionary of dataframes from all the csv files in the folder
# and then returning to the original folder.

@contextmanager
def gen_list(file_path, extension):
    f = os.chdir(file_path)
    yield f
    os.chdir(script_path)
    
with gen_list(file_path, extension) as f:
    csv_list = glob.glob(r"*{}".format(extension))
    dataframe_names = []
    df_dict = {}
    iterator = 0
    for i in csv_list:
        dataframe_names.append(i[:-4].replace(".", "_")) # Saves the names of the csv files minus .csv and also changes the periods in the name to underscores.
    for j in dataframe_names:
        df_dict[j] = pd.read_csv(csv_list[iterator], sep=None, engine="python") # Key is the name of the csv file formatted as above.
        iterator += 1

# Saves the column names of processed_cleveland and sets the other datasets column names to the same.     

column_names = list(df_dict["processed_cleveland"].columns)

for df, names in df_dict.items():
    names.columns = column_names
    
# Function to set the dtypes as specified in the earlier lists. The function increases the amount of null values for some reason,
# most probably cause by the to_numeric function's errors parameter. TODO: Find out which values is changing to nulls.

def set_dtypes(df_dict):
    for df, name in df_dict.items():
        for column in name.columns:
            if column in categorical_features:
                name[column] = name[column].astype("category", copy=False)
            elif column in boolean_features:
                name[column] = name[column].astype("Int64", copy=False)
            elif column in float_features:
                name[column] = pd.to_numeric(name[column], downcast="float", errors="coerce")
            elif column in integer_features:
                name[column] = pd.to_numeric(name[column], downcast="float", errors="coerce")
                name[column] = name[column].astype("Int64", copy=False)
    return df_dict

In [4]:
print(df_dict.keys())

dict_keys(['processed_cleveland', 'processed_hungarian', 'processed_switzerland', 'processed_va'])


In [5]:
# Imputing nulls instead of "?" for all dataframes in the dictionary.
# TODO: print name of the dataframe before isnull list.

for name in df_dict.keys():
    print(name)
    print(df_dict[name].isnull().sum().sum())

processed_cleveland
0
processed_hungarian
0
processed_switzerland
0
processed_va
0


In [6]:
for name in df_dict:
    df_dict[name].replace(to_replace="?", value=np.nan, inplace=True)

for name in df_dict:
    print(name)
    print(df_dict[name].isnull().sum().sum())

processed_cleveland
6
processed_hungarian
782
processed_switzerland
273
processed_va
698


In [7]:
for df, name in df_dict.items():
    print(df)
    for i in name.columns:
        print(i)
        print(name[i].unique())

processed_cleveland
age
[63. 67. 37. 41. 56. 62. 57. 53. 44. 52. 48. 54. 49. 64. 58. 60. 50. 66.
 43. 40. 69. 59. 42. 55. 61. 65. 71. 51. 46. 45. 39. 68. 47. 34. 35. 29.
 70. 77. 38. 74. 76.]
sex
[1. 0.]
cp
[1. 4. 3. 2.]
trestbps
[145. 160. 120. 130. 140. 172. 150. 110. 132. 117. 135. 112. 105. 124.
 125. 142. 128. 170. 155. 104. 180. 138. 108. 134. 122. 115. 118. 100.
 200.  94. 165. 102. 152. 101. 126. 174. 148. 178. 158. 192. 129. 144.
 123. 136. 146. 106. 156. 154. 114. 164.]
chol
[233. 286. 229. 250. 204. 236. 268. 354. 254. 203. 192. 294. 256. 263.
 199. 168. 239. 275. 266. 211. 283. 284. 224. 206. 219. 340. 226. 247.
 167. 230. 335. 234. 177. 276. 353. 243. 225. 302. 212. 330. 175. 417.
 197. 198. 290. 253. 172. 273. 213. 305. 216. 304. 188. 282. 185. 232.
 326. 231. 269. 267. 248. 360. 258. 308. 245. 270. 208. 264. 321. 274.
 325. 235. 257. 164. 141. 252. 255. 201. 222. 260. 182. 303. 265. 309.
 307. 249. 186. 341. 183. 407. 217. 288. 220. 209. 227. 261. 174. 281.
 221. 205. 24

In [8]:
for df, name in df_dict.items():
    print(df)
    print(name.isnull().sum().sum())

processed_cleveland
6
processed_hungarian
782
processed_switzerland
273
processed_va
698


In [9]:
df_dict = set_dtypes(df_dict)

In [13]:
for df, name in df_dict.items():
    print(df)
    print(name.isnull().sum().sum())

processed_cleveland
6
processed_hungarian
782
processed_switzerland
273
processed_va
698


In [14]:
for df, names in df_dict.items():
    print(names.dtypes)

age            Int64
sex         category
cp          category
trestbps       Int64
chol           Int64
fbs            Int64
restecg     category
thalach        Int64
exang          Int64
oldpeak      float32
slope       category
ca             Int64
thal        category
num            int64
dtype: object
age            Int64
sex         category
cp          category
trestbps       Int64
chol           Int64
fbs            Int64
restecg     category
thalach        Int64
exang          Int64
oldpeak      float32
slope       category
ca             Int64
thal        category
num            int64
dtype: object
age            Int64
sex         category
cp          category
trestbps       Int64
chol           Int64
fbs            Int64
restecg     category
thalach        Int64
exang          Int64
oldpeak      float32
slope       category
ca             Int64
thal        category
num            int64
dtype: object
age            Int64
sex         category
cp          category
trestbps     

In [11]:
with gen_list(file_path, extension) as f:
    for df, name in df_dict.items():
        name.to_csv( f"./Cleaned data/cleaned_{df}.csv", header=name.columns, index=False)